In [1]:
from sqlalchemy import create_engine
import pandas as pd

# Create engine using SQLAlchemy for both databases
engine_cm = create_engine('mssql+pyodbc://ABDALLAH\\MSSQLSERVER2/CustomerManagement?trusted_connection=yes&driver=SQL+Server')
engine_dwh = create_engine('mssql+pyodbc://ABDALLAH\\MSSQLSERVER2/CustomerManagementDWH?trusted_connection=yes&driver=SQL+Server')

In [2]:
import pandas as pd

# Extract data from Customers table in CustomerManagement DB
query_extract_customers = "SELECT * FROM Customers"
customers_df = pd.read_sql(query_extract_customers, engine_cm)


In [5]:
# Transform data: remove columns that are not present in Customers_Dim
customers_df = customers_df.drop(columns=['ShippingAddress', 'BillingAddress'])

# Rename or transform other columns if needed
customers_df = customers_df.rename(columns={
    'CustomerID': 'CustomerID',
    'FullName': 'FullName',
    'Email': 'Email',
    'PhoneNumber': 'PhoneNumber',
    'DateOfBirth': 'DateOfBirth',
    'LoyaltyPoints': 'LoyaltyPoints',
    'Preferences': 'PreferredCategories'  # Assuming preferences map to this
})

# Handle any null values or necessary transformations
customers_df['LoyaltyPoints'].fillna(0, inplace=True)


In [6]:
# Load data into Customers_Dim table in CustomerManagementDWH
customers_df.to_sql('Customers_Dim', engine_dwh, if_exists='append', index=False)


-1

In [14]:
# Extract data from Payments_Dim table in CustomerManagementDWH
query_extract_payments = "SELECT PaymentID, PaymentMethod FROM Payments_Dim"
payments_dim_df = pd.read_sql(query_extract_payments, engine_dwh)

# Create a mapping dictionary
payment_method_to_id = dict(zip(payments_dim_df['PaymentMethod'], payments_dim_df['PaymentID']))

# Define the get_payment_id function
def get_payment_id(payment_method):
    return payment_method_to_id.get(payment_method, None)  # Returns None if payment method is not found


In [15]:
# Apply the get_payment_id function to the PaymentMethod column in orders_df
orders_df['PaymentID'] = orders_df['PaymentMethod'].apply(lambda x: get_payment_id(x))

# Drop the PaymentMethod column as it is no longer needed
orders_df = orders_df.drop(columns=['PaymentMethod'])


In [17]:
# Extract Date_Dim from the Data Warehouse
query_extract_date_dim = "SELECT DateID, Date FROM Date_Dim"
date_dim_df = pd.read_sql(query_extract_date_dim, engine_dwh)


In [18]:
# Create a mapping from Date to DateID
date_mapping = dict(zip(date_dim_df['Date'], date_dim_df['DateID']))

# Map OrderDate to DateID
orders_df['OrderDateID'] = orders_df['OrderDateID'].apply(lambda x: date_mapping.get(x.date(), None))


In [22]:
# Extract Address_Dim from Data Warehouse
query_extract_address_dim = "SELECT AddressID, FullAddress FROM Address_Dim"
address_dim_df = pd.read_sql(query_extract_address_dim, engine_dwh)


In [23]:
# Create a mapping from FullAddress to AddressID
address_mapping = dict(zip(address_dim_df['FullAddress'], address_dim_df['AddressID']))

# Map ShippingAddress and BillingAddress to AddressID
orders_df['ShippingAddressID'] = orders_df['ShippingAddressID'].apply(lambda x: address_mapping.get(x, None))
orders_df['BillingAddressID'] = orders_df['BillingAddressID'].apply(lambda x: address_mapping.get(x, None))


In [24]:
print(orders_df[['ShippingAddressID', 'BillingAddressID']].isnull().sum())  # Should be 0 if all addresses were mapped


ShippingAddressID    100
BillingAddressID     100
dtype: int64


In [25]:
# Load data into Orders_Fact table in CustomerManagementDWH
orders_df.to_sql('Orders_Fact', engine_dwh, if_exists='append', index=False)


-1

In [31]:
# Extract data from Payments table in the CustomerManagement database
payments_df = pd.read_sql("SELECT PaymentID, PaymentMethod, PaymentDate, Amount FROM Payments", engine_cm)
print(payments_df.head())  # Display first few rows to check the data


   PaymentID  PaymentMethod         PaymentDate  Amount
0          1         PayPal 2024-07-24 17:59:23  323.32
1          2         PayPal 2023-12-09 17:59:23  200.32
2          3  Bank Transfer 2024-07-05 17:59:23  323.32
3          4     Debit Card 2024-03-06 17:59:23  304.07
4          5     Debit Card 2024-03-08 17:59:23  428.00


In [32]:
# Perform any necessary transformations (e.g., converting PaymentDate format, mapping PaymentMethods, handling missing values)

# Example transformation: Convert PaymentDate to a proper date format
payments_df['PaymentDate'] = pd.to_datetime(payments_df['PaymentDate'])

# Ensure that all necessary fields are present and have valid values
payments_df = payments_df.dropna(subset=['PaymentID', 'PaymentMethod', 'PaymentDate', 'Amount'])

print(payments_df.dtypes)  # Check data types to ensure they're correct


PaymentID                 int64
PaymentMethod            object
PaymentDate      datetime64[ns]
Amount                  float64
dtype: object


In [33]:
# Load data into Payments_Dim table in CustomerManagementDWH
payments_df.to_sql('Payments_Dim', engine_dwh, if_exists='append', index=False)


-1

In [34]:
# Extract data from Products, Categories, and Brands tables in CustomerManagement
products_query = """
SELECT 
    p.ProductID, 
    p.ProductName, 
    c.CategoryName AS Category, 
    b.BrandName AS Brand, 
    p.Price, 
    p.Rating
FROM Products p
LEFT JOIN Categories c ON p.CategoryID = c.CategoryID
LEFT JOIN Brands b ON p.BrandID = b.BrandID
"""

products_df = pd.read_sql(products_query, engine_cm)
print(products_df.head())  # Display first few rows to check the data


   ProductID ProductName    Category     Brand  Price  Rating
0          1   Product 1  Category 9   Brand 4  26.07     4.0
1          2   Product 2  Category 2   Brand 2  23.00     3.8
2          3   Product 3  Category 7  Brand 10  60.05     2.5
3          4   Product 4  Category 3   Brand 1  58.66     3.8
4          5   Product 5  Category 5   Brand 1  92.61     4.8


In [35]:
# Perform any necessary transformations (e.g., handling missing values, data type conversion)
# For example, ensuring 'Price' is numeric and 'Rating' is within a valid range

# Drop any rows with missing mandatory values
products_df = products_df.dropna(subset=['ProductID', 'ProductName', 'Category', 'Brand', 'Price'])

# Check the data types to ensure everything is correct
print(products_df.dtypes)


ProductID        int64
ProductName     object
Category        object
Brand           object
Price          float64
Rating         float64
dtype: object


In [36]:
# Load data into Products_Dim table in CustomerManagementDWH
products_df.to_sql('Products_Dim', engine_dwh, if_exists='append', index=False)


-1

In [37]:
import pandas as pd

# Define the start and end date for the date dimension
start_date = '2020-01-01'
end_date = '2025-12-31'

# Create a date range
date_range = pd.date_range(start=start_date, end=end_date)

# Create a DataFrame for the Date_Dim table
date_dim_df = pd.DataFrame({
    'Date': date_range,
    'Day': date_range.day,
    'Month': date_range.month,
    'Year': date_range.year,
    'Quarter': date_range.quarter,
    'Weekday': date_range.strftime('%A')  # Convert weekday to a string like 'Monday'
})

# Create a unique DateID (YYYYMMDD format)
date_dim_df['DateID'] = date_dim_df['Date'].dt.strftime('%Y%m%d').astype(int)

# Reorder columns to match the target schema
date_dim_df = date_dim_df[['DateID', 'Date', 'Day', 'Month', 'Year', 'Quarter', 'Weekday']]

print(date_dim_df.head())  # Check the first few rows


     DateID       Date  Day  Month  Year  Quarter    Weekday
0  20200101 2020-01-01    1      1  2020        1  Wednesday
1  20200102 2020-01-02    2      1  2020        1   Thursday
2  20200103 2020-01-03    3      1  2020        1     Friday
3  20200104 2020-01-04    4      1  2020        1   Saturday
4  20200105 2020-01-05    5      1  2020        1     Sunday


In [38]:
# Load data into Date_Dim table in CustomerManagementDWH
date_dim_df.to_sql('Date_Dim', engine_dwh, if_exists='append', index=False)


-1

In [39]:
# Extract data from Customers and Orders for addresses
customers_addresses_df = pd.read_sql('SELECT DISTINCT ShippingAddress, BillingAddress FROM Customers', engine_cm)
orders_addresses_df = pd.read_sql('SELECT DISTINCT ShippingAddress, BillingAddress FROM Orders', engine_cm)

# Concatenate the two DataFrames to get all distinct addresses
addresses_df = pd.concat([customers_addresses_df, orders_addresses_df]).drop_duplicates().reset_index(drop=True)

print(addresses_df.head())  # Preview the data


                                     ShippingAddress  \
0  00655 Debra Prairie Suite 552, Port Anthony, P...   
1        0228 Morrow Knolls\nSouth Kathryn, DC 26571   
2  033 Holly Fields Suite 287, West Donnaview, KY...   
3  0346 Johnson Stream Apt. 929, North Jamesfort,...   
4          0393 Morgan Turnpike\nTerryfort, MI 07625   

                                      BillingAddress  
0                           USNV Clark, FPO AE 98671  
1            7204 Bell Park\nChristinafort, TN 81193  
2  34546 Flynn Valleys Suite 970, South Jillian, ...  
3  73981 Carlos Squares Apt. 979, Josephchester, ...  
4                   Unit 2422 Box 0422\nDPO AA 93174  


In [40]:
# Splitting the address into components (assuming they follow a specific format)
# Example: "123 Main St Suite 100\nSpringfield, IL 62704"

# Define a helper function to split the address into parts
def split_address(address):
    try:
        parts = address.split('\n')  # Split by newline first
        full_address = parts[0]
        city_state_zip = parts[1].split(',')  # Split city and state+zip
        city = city_state_zip[0]
        state_zip = city_state_zip[1].split(' ')  # Split state and zip
        state = state_zip[0]
        postal_code = state_zip[1]
        return pd.Series([full_address, city, state, 'USA', postal_code])  # Assuming the country is USA
    except:
        return pd.Series([address, None, None, None, None])  # In case of any issues, fill with None

# Apply the split function to both Shipping and Billing addresses
addresses_df[['FullAddress', 'City', 'State', 'Country', 'PostalCode']] = addresses_df['ShippingAddress'].apply(split_address)

# Drop the original ShippingAddress and BillingAddress columns
addresses_df = addresses_df.drop(columns=['ShippingAddress', 'BillingAddress'])

# Add a unique AddressID
addresses_df['AddressID'] = range(1, len(addresses_df) + 1)

print(addresses_df.head())  # Preview the transformed data


                                         FullAddress           City State  \
0  00655 Debra Prairie Suite 552, Port Anthony, P...           None  None   
1                                 0228 Morrow Knolls  South Kathryn         
2  033 Holly Fields Suite 287, West Donnaview, KY...           None  None   
3  0346 Johnson Stream Apt. 929, North Jamesfort,...           None  None   
4                               0393 Morgan Turnpike      Terryfort         

  Country PostalCode  AddressID  
0    None       None          1  
1     USA         DC          2  
2    None       None          3  
3    None       None          4  
4     USA         MI          5  


In [41]:
# Load data into Address_Dim table in CustomerManagementDWH
addresses_df.to_sql('Address_Dim', engine_dwh, if_exists='append', index=False)


-1